In [3]:
!pip install fasttext

     |████████████████████████████████| 68 kB 2.3 MB/s eta 0:00:011
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3144377 sha256=6b60a60b33b1dc87198461b0ddec4ca2539ec37050bc9f0285201fcea7152faa
  Stored in directory: /root/.cache/pip/wheels/c3/5c/d0/4a725c6ee7df3267d818d3bc9d89bb173b94832f2b9eca6368
Successfully built fasttext
You should consider upgrading via the '/usr/bin/python3.6 -m pip install --upgrade pip' command.


In [1]:
import numpy as np
import pandas as pd
import string
import datetime
import matplotlib.pyplot as plt
import os
import gc
import fasttext.util
import random
import joblib
from itertools import product

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight

from scipy.spatial.distance import jensenshannon
import scipy.io as sio
from scipy.special import rel_entr, kl_div
from scipy.stats import entropy, ks_2samp
from scipy.spatial.distance import jensenshannon
from scipy.special import rel_entr

from tensorflow.keras.layers import Input, LSTM, Bidirectional, Dense, Embedding, Flatten, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow import keras
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import tensorflow as tf

In [2]:
ft = fasttext.load_model('cc.en.300.bin')
samples_number = 7

In [3]:
df_accuracy = pd.DataFrame(np.zeros([2015-2007, (2015-2007)*samples_number]), index=range(2007, 2015), columns=pd.MultiIndex.from_product([range(2007, 2015), range(samples_number)]))
df_f1 = df_accuracy.copy()
df_accuracy

2007                               2008            ... 2013            \
        0    1    2    3    4    5    6    0    1    2  ...    4    5    6   
2007  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2008  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2009  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2010  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2011  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2012  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2013  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2014  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

     2014                                
        0    1    2    3    4    5    6  
2007  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2008  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2009  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2010  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2011  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2012  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2013  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2014  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[8 rows x 56 columns]

In [4]:
def Generator(X_data, y_data, batch_size):

    samples_per_epoch = X_data.shape[0]
    number_of_batches = samples_per_epoch/batch_size
    counter=0

    while 1:

        X_batch = np.array(X_data[batch_size*counter:batch_size*(counter+1)])
        y_batch = np.array(y_data[batch_size*counter:batch_size*(counter+1)])
        
        counter += 1
        yield X_batch,y_batch
        
        if counter >= number_of_batches:
            counter = 0

def ReadData(year, index):
    data = pd.read_csv('data/{0}_{1}.csv'.format(year, index), index_col=[0])['words'].astype(str)
    labels = (pd.read_csv('data/{0}_{1}.csv'.format(year, index), index_col=[0])['label'])
    return data, labels

def FormTrainData(year_train, test_index, samples_number):
    train_data = pd.Series(dtype=pd.StringDtype())
    train_labels = pd.Series(dtype=int)
    for train_index in range(samples_number):
        if test_index != train_index:
            data, labels = ReadData(year_train, train_index)
            train_data = train_data.append(data)
            train_labels = train_labels.append(labels)
    return train_data, train_labels


def BuildEmbeddingMatrix(nb_words, word_index):
    embedding_matrix = np.zeros((nb_words, 300))
    for word, i in word_index.items():
        if i >= nb_words:
            continue
        try:
            embedding_vector = ft.get_word_vector(word)
        except KeyError:
            embedding_vector = None
        if (embedding_vector is not None) and len(embedding_vector) > 0:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

def f1(y_true, y_pred):
    y_pred = tf.reduce_max(y_pred, axis=-1, keepdims=True)
    tp = tf.reduce_sum(y_true*y_pred, axis=0)
    tn = tf.reduce_sum((1-y_true)*(1-y_pred), axis=0)
    fp = tf.reduce_sum((1-y_true)*y_pred, axis=0)
    fn = tf.reduce_sum(y_true*(1-y_pred), axis=0)

    p = tp / (tp + fp + np.finfo(float).eps)
    r = tp / (tp + fn + np.finfo(float).eps)

    f1 = 2*p*r / (p+r+np.finfo(float).eps)
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return tf.reduce_mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = tf.reduce_sum(y_true*y_pred, axis=0)
    tn = tf.reduce_sum((1-y_true)*(1-y_pred), axis=0)
    fp = tf.reduce_sum((1-y_true)*y_pred, axis=0)
    fn = tf.reduce_sum(y_true*(1-y_pred), axis=0)

    p = tp / (tp + fp + np.finfo(float).eps)
    r = tp / (tp + fn + np.finfo(float).eps)

    f1 = 2*p*r / (p+r+np.finfo(float).eps)
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - tf.reduce_mean(f1)

In [103]:
def test_model(model, loss_function, class_weight, epochs):    
     
    model.compile(
        loss=loss_function,
        optimizer='adam',
        metrics=[f1, 'accuracy'],
    )
    model.summary()
    
    batch_size = 16
    gen = Generator(train_data, to_categorical(train_labels), batch_size)
    history = model.fit(
                gen,
                class_weight=class_weight,
                epochs=epochs,
                steps_per_epoch = train_data.shape[0]//batch_size,
    #                 validation_data = Generator(test_data, to_categorical(test_labels),batch_size*2),
    #                 validation_steps = (test_data.shape[0]/batch_size)
            )

    loss, fbeta_score, accuracy = model.evaluate(test_data, to_categorical(test_labels), verbose=False)
    y_pred = np.argmax(model.predict(test_data), axis=1)
    fbeta_score = f1_score(y_pred, test_labels, average='macro')
    df_f1.loc[year_train][year_test][test_index] = fbeta_score
    df_accuracy.loc[year_train][year_test][test_index] = accuracy

    print("Train in {0}, test in {1}, test sample {2}\nf1_score = {3}\naccuracy = {4}".format(year_train, year_test, test_index, fbeta_score, accuracy))
    print('\nConfusion Matrix')
    print(confusion_matrix(test_labels, y_pred))
    print('\nClassification Report')
    print(classification_report(test_labels, y_pred))


### Один год, 3 класса, f loss

In [41]:
year_train=2007
year_test=2007
test_index=6

vectorizer = TextVectorization(max_tokens=10000, output_sequence_length=300)
train_data, train_labels = FormTrainData(year_train, test_index, 7)
train_labels = train_labels.apply(lambda x: 1 if x == 3 else (0 if x < 3 else 2))
vectorizer.adapt(train_data.values)
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))
nb_words = len(voc)+2
train_data = vectorizer(np.array([[s] for s in train_data])).numpy()
test_data, test_labels = ReadData(year_test, test_index)
test_labels = test_labels.apply(lambda x: 1 if x == 3 else (0 if x < 3 else 2))
test_data = vectorizer(np.array([[s] for s in test_data])).numpy()
embedding_matrix = BuildEmbeddingMatrix(nb_words, word_index)

print(nb_words, train_data.shape[1])

10001 300


In [42]:
class_weights = class_weight.compute_class_weight('balanced', np.unique(test_labels), test_labels)
class_weights

array([2.63296472, 3.97772474, 0.42215468])

In [43]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(64))(x)
outputs = Dense(3, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function=f1_loss, class_weight=[1, 1, 1], epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_23 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_29 (Bidirectio (None, 128)               186880    
_________________________________________________________________
dense_27 (Dense)             (None, 3)                 387       
Total params: 3,187,567
Trainable params: 187,267
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 1875 steps
Epoch 1/10
1875/1875 [==============================] - 34s 18ms/step - loss: 0.6721 - f1: 0.3880 - accuracy: 0.7346
Epoch 2/10
1875/1875 [==============================] - 33s 18ms/step - loss: 0.

In [44]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(128))(x)
outputs = Dense(3, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function=f1_loss, class_weight=[1, 1, 1], epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_24 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_30 (Bidirectio (None, 256)               439296    
_________________________________________________________________
dense_28 (Dense)             (None, 3)                 771       
Total params: 3,440,367
Trainable params: 440,067
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 1875 steps
Epoch 1/10
1875/1875 [==============================] - 44s 23ms/step - loss: 0.6736 - f1: 0.3890 - accuracy: 0.6893
Epoch 2/10
1875/1875 [==============================] - 44s 23ms/step - loss: 0.

In [45]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = LSTM(128)(x)
outputs = Dense(3, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function=f1_loss, class_weight=[1, 1, 1], epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_26 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_25 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
lstm_33 (LSTM)               (None, 128)               219648    
_________________________________________________________________
dense_29 (Dense)             (None, 3)                 387       
Total params: 3,220,335
Trainable params: 220,035
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 1875 steps
Epoch 1/10
1875/1875 [==============================] - 24s 13ms/step - loss: 0.6825 - f1: 0.3785 - accuracy: 0.7833
Epoch 2/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.

In [46]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(64))(x)
x = Dense(32, activation='relu')(x)
outputs = Dense(3, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function=f1_loss, class_weight=[1, 1, 1], epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_27 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_26 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_31 (Bidirectio (None, 128)               186880    
_________________________________________________________________
dense_30 (Dense)             (None, 32)                4128      
_________________________________________________________________
dense_31 (Dense)             (None, 3)                 99        
Total params: 3,191,407
Trainable params: 191,107
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 1875 steps
Epoch 1/10
1875/1875 [==============================] - 36s 19ms/

In [47]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(128))(x)
x = Dense(64, activation='relu')(x)
outputs = Dense(3, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function=f1_loss, class_weight=[1, 1, 1], epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_28 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_27 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_32 (Bidirectio (None, 256)               439296    
_________________________________________________________________
dense_32 (Dense)             (None, 64)                16448     
_________________________________________________________________
dense_33 (Dense)             (None, 3)                 195       
Total params: 3,456,239
Trainable params: 455,939
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 1875 steps
Epoch 1/10
1875/1875 [==============================] - 44s 24ms/

In [48]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Bidirectional(LSTM(32))(x)
outputs = Dense(3, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function=f1_loss, class_weight=[1, 1, 1], epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_28 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_33 (Bidirectio (None, None, 256)         439296    
_________________________________________________________________
bidirectional_34 (Bidirectio (None, 64)                73984     
_________________________________________________________________
dense_34 (Dense)             (None, 3)                 195       
Total params: 3,513,775
Trainable params: 513,475
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 1875 steps
Epoch 1/10
1875/1875 [==============================] - 83s 44ms/

In [49]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Bidirectional(LSTM(64))(x)
outputs = Dense(3, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function=f1_loss, class_weight=[1, 1, 1], epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_30 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_29 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_35 (Bidirectio (None, None, 256)         439296    
_________________________________________________________________
bidirectional_36 (Bidirectio (None, 128)               164352    
_________________________________________________________________
dense_35 (Dense)             (None, 3)                 387       
Total params: 3,604,335
Trainable params: 604,035
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 1875 steps
Epoch 1/10
1875/1875 [==============================] - 85s 45ms/

In [50]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Bidirectional(LSTM(32, return_sequences=True))(x)
x = Bidirectional(LSTM(8, return_sequences=False))(x)
outputs = Dense(3, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function=f1_loss, class_weight=[1, 1, 1], epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_31 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_30 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_37 (Bidirectio (None, None, 256)         439296    
_________________________________________________________________
bidirectional_38 (Bidirectio (None, None, 64)          73984     
_________________________________________________________________
bidirectional_39 (Bidirectio (None, 16)                4672      
_________________________________________________________________
dense_36 (Dense)             (None, 3)                 51        
Total params: 3,518,303
Trainable params: 518,003
Non-trainable params: 3,000,300
_____________________________________________

### Один год, 3 класса, weighted categorical_crossentropy loss

In [51]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(64))(x)
outputs = Dense(3, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function='categorical_crossentropy', class_weight=class_weight, epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_32 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_31 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_40 (Bidirectio (None, 128)               186880    
_________________________________________________________________
dense_37 (Dense)             (None, 3)                 387       
Total params: 3,187,567
Trainable params: 187,267
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 1875 steps
Epoch 1/10
1875/1875 [==============================] - 35s 19ms/step - loss: 0.6573 - f1: 0.3715 - accuracy: 0.7893
Epoch 2/10
1875/1875 [==============================] - 34s 18ms/step - loss: 0.

In [52]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(64))(x)
outputs = Dense(3, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function='categorical_crossentropy', class_weight=[1,1,1], epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_33 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_32 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_41 (Bidirectio (None, 128)               186880    
_________________________________________________________________
dense_38 (Dense)             (None, 3)                 387       
Total params: 3,187,567
Trainable params: 187,267
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 1875 steps
Epoch 1/10
1875/1875 [==============================] - 35s 19ms/step - loss: 0.6605 - f1: 0.3719 - accuracy: 0.7889
Epoch 2/10
1875/1875 [==============================] - 34s 18ms/step - loss: 0.

In [53]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(128))(x)
outputs = Dense(3, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function='categorical_crossentropy', class_weight=class_weight, epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_34 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_33 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_42 (Bidirectio (None, 256)               439296    
_________________________________________________________________
dense_39 (Dense)             (None, 3)                 771       
Total params: 3,440,367
Trainable params: 440,067
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 1875 steps
Epoch 1/10
1875/1875 [==============================] - 44s 23ms/step - loss: 0.6635 - f1: 0.3724 - accuracy: 0.7893
Epoch 2/10
1875/1875 [==============================] - 44s 23ms/step - loss: 0.

In [54]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = LSTM(128)(x)
outputs = Dense(3, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function='categorical_crossentropy', class_weight=class_weight, epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_35 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_34 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
lstm_46 (LSTM)               (None, 128)               219648    
_________________________________________________________________
dense_40 (Dense)             (None, 3)                 387       
Total params: 3,220,335
Trainable params: 220,035
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 1875 steps
Epoch 1/10
1875/1875 [==============================] - 24s 13ms/step - loss: 0.6694 - f1: 0.3729 - accuracy: 0.7894
Epoch 2/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.

In [55]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(64))(x)
x = Dense(32, activation='relu')(x)
outputs = Dense(3, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function='categorical_crossentropy', class_weight=class_weight, epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_36 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_35 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_43 (Bidirectio (None, 128)               186880    
_________________________________________________________________
dense_41 (Dense)             (None, 32)                4128      
_________________________________________________________________
dense_42 (Dense)             (None, 3)                 99        
Total params: 3,191,407
Trainable params: 191,107
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 1875 steps
Epoch 1/10
1875/1875 [==============================] - 36s 19ms/

In [56]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(128))(x)
x = Dense(64, activation='relu')(x)
outputs = Dense(3, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function='categorical_crossentropy', class_weight=class_weight, epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_37 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_36 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_44 (Bidirectio (None, 256)               439296    
_________________________________________________________________
dense_43 (Dense)             (None, 64)                16448     
_________________________________________________________________
dense_44 (Dense)             (None, 3)                 195       
Total params: 3,456,239
Trainable params: 455,939
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 1875 steps
Epoch 1/10
1875/1875 [==============================] - 44s 24ms/

In [57]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Bidirectional(LSTM(32))(x)
outputs = Dense(3, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function='categorical_crossentropy', class_weight=class_weight, epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_38 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_37 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_45 (Bidirectio (None, None, 256)         439296    
_________________________________________________________________
bidirectional_46 (Bidirectio (None, 64)                73984     
_________________________________________________________________
dense_45 (Dense)             (None, 3)                 195       
Total params: 3,513,775
Trainable params: 513,475
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 1875 steps
Epoch 1/10
1875/1875 [==============================] - 82s 44ms/

In [58]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Bidirectional(LSTM(64))(x)
outputs = Dense(3, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function='categorical_crossentropy', class_weight=class_weight, epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_39 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_38 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_47 (Bidirectio (None, None, 256)         439296    
_________________________________________________________________
bidirectional_48 (Bidirectio (None, 128)               164352    
_________________________________________________________________
dense_46 (Dense)             (None, 3)                 387       
Total params: 3,604,335
Trainable params: 604,035
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 1875 steps
Epoch 1/10
1875/1875 [==============================] - 84s 45ms/

In [59]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Bidirectional(LSTM(32, return_sequences=True))(x)
x = Bidirectional(LSTM(8, return_sequences=False))(x)
outputs = Dense(3, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function='categorical_crossentropy', class_weight=class_weight, epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_40 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_39 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_49 (Bidirectio (None, None, 256)         439296    
_________________________________________________________________
bidirectional_50 (Bidirectio (None, None, 64)          73984     
_________________________________________________________________
bidirectional_51 (Bidirectio (None, 16)                4672      
_________________________________________________________________
dense_47 (Dense)             (None, 3)                 51        
Total params: 3,518,303
Trainable params: 518,003
Non-trainable params: 3,000,300
_____________________________________________

### Два года, 3 класса, weighted categorical_crossentropy loss

In [124]:
years_train=[2007,2008]
years_test=[2007, 2008]
test_index=6

vectorizer = TextVectorization(max_tokens=10000, output_sequence_length=300)

train_data, train_labels = FormTrainData(years_train[0], test_index, 7)
test_data, test_labels = ReadData(years_test[0], test_index)
train_data_2, train_labels_2 = FormTrainData(years_train[1], test_index, 7)
test_data_2, test_labels_2 = ReadData(years_test[1], test_index)
train_data = pd.concat((train_data,train_data_2),axis=0)
train_labels = pd.concat((train_labels,train_labels_2),axis=0)
test_data = pd.concat((test_data,test_data_2),axis=0)
test_labels = pd.concat((test_labels,test_labels_2),axis=0)


test_labels = test_labels.apply(lambda x: 1 if x == 3 else (0 if x < 3 else 2))
train_labels = train_labels.apply(lambda x: 1 if x == 3 else (0 if x < 3 else 2))
class_weights = class_weight.compute_class_weight('balanced', np.unique(test_labels), test_labels)

vectorizer.adapt(train_data.values)
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))
nb_words = len(voc)+2
train_data = vectorizer(np.array([[s] for s in train_data])).numpy()
test_data = vectorizer(np.array([[s] for s in test_data])).numpy()
embedding_matrix = BuildEmbeddingMatrix(nb_words, word_index)
print(nb_words, train_data.shape[1], class_weights)

10001 300 [2.66880171 3.94944708 0.42156739]


In [108]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(64))(x)
outputs = Dense(3, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function='categorical_crossentropy', class_weight=class_weight, epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_55 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_54 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_65 (Bidirectio (None, 128)               186880    
_________________________________________________________________
dense_63 (Dense)             (None, 3)                 387       
Total params: 3,187,567
Trainable params: 187,267
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 3750 steps
Epoch 1/10
3750/3750 [==============================] - 66s 18ms/step - loss: 0.6464 - f1: 0.3737 - accuracy: 0.7907
Epoch 2/10
3750/3750 [==============================] - 66s 18ms/step - loss: 0.

In [109]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(64))(x)
outputs = Dense(3, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function='categorical_crossentropy', class_weight=[1,1,1], epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_56 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_55 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_66 (Bidirectio (None, 128)               186880    
_________________________________________________________________
dense_64 (Dense)             (None, 3)                 387       
Total params: 3,187,567
Trainable params: 187,267
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 3750 steps
Epoch 1/10
3750/3750 [==============================] - 69s 18ms/step - loss: 0.6451 - f1: 0.3738 - accuracy: 0.7907
Epoch 2/10
3750/3750 [==============================] - 68s 18ms/step - loss: 0.

In [110]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(128))(x)
outputs = Dense(3, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function='categorical_crossentropy', class_weight=class_weight, epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_57 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_56 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_67 (Bidirectio (None, 256)               439296    
_________________________________________________________________
dense_65 (Dense)             (None, 3)                 771       
Total params: 3,440,367
Trainable params: 440,067
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 3750 steps
Epoch 1/10
3750/3750 [==============================] - 88s 23ms/step - loss: 0.6446 - f1: 0.3734 - accuracy: 0.7907
Epoch 2/10
3750/3750 [==============================] - 88s 23ms/step - loss: 0.

In [111]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = LSTM(128)(x)
outputs = Dense(3, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function='categorical_crossentropy', class_weight=class_weight, epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_58 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_57 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
lstm_73 (LSTM)               (None, 128)               219648    
_________________________________________________________________
dense_66 (Dense)             (None, 3)                 387       
Total params: 3,220,335
Trainable params: 220,035
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 3750 steps
Epoch 1/10
3750/3750 [==============================] - 47s 12ms/step - loss: 0.6611 - f1: 0.3750 - accuracy: 0.7907
Epoch 2/10
3750/3750 [==============================] - 46s 12ms/step - loss: 0.

In [112]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(64))(x)
x = Dense(32, activation='relu')(x)
outputs = Dense(3, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function='categorical_crossentropy', class_weight=class_weight, epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_59 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_58 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_68 (Bidirectio (None, 128)               186880    
_________________________________________________________________
dense_67 (Dense)             (None, 32)                4128      
_________________________________________________________________
dense_68 (Dense)             (None, 3)                 99        
Total params: 3,191,407
Trainable params: 191,107
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 3750 steps
Epoch 1/10
3750/3750 [==============================] - 71s 19ms/

In [113]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(128))(x)
x = Dense(64, activation='relu')(x)
outputs = Dense(3, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function='categorical_crossentropy', class_weight=class_weight, epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_60 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_59 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_69 (Bidirectio (None, 256)               439296    
_________________________________________________________________
dense_69 (Dense)             (None, 64)                16448     
_________________________________________________________________
dense_70 (Dense)             (None, 3)                 195       
Total params: 3,456,239
Trainable params: 455,939
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 3750 steps
Epoch 1/10
3750/3750 [==============================] - 89s 24ms/

In [125]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Bidirectional(LSTM(32))(x)
outputs = Dense(3, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function='categorical_crossentropy', class_weight=class_weight, epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_69 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_68 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_82 (Bidirectio (None, None, 256)         439296    
_________________________________________________________________
bidirectional_83 (Bidirectio (None, 64)                73984     
_________________________________________________________________
dense_79 (Dense)             (None, 3)                 195       
Total params: 3,513,775
Trainable params: 513,475
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 3750 steps
Epoch 1/10
3750/3750 [==============================] - 155s 41ms

In [126]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Bidirectional(LSTM(64))(x)
outputs = Dense(3, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function='categorical_crossentropy', class_weight=class_weight, epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_70 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_69 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_84 (Bidirectio (None, None, 256)         439296    
_________________________________________________________________
bidirectional_85 (Bidirectio (None, 128)               164352    
_________________________________________________________________
dense_80 (Dense)             (None, 3)                 387       
Total params: 3,604,335
Trainable params: 604,035
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 3750 steps
Epoch 1/10
3750/3750 [==============================] - 164s 44ms

In [127]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Bidirectional(LSTM(32, return_sequences=True))(x)
x = Bidirectional(LSTM(8, return_sequences=False))(x)
outputs = Dense(3, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function='categorical_crossentropy', class_weight=class_weight, epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_71 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_70 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_86 (Bidirectio (None, None, 256)         439296    
_________________________________________________________________
bidirectional_87 (Bidirectio (None, None, 64)          73984     
_________________________________________________________________
bidirectional_88 (Bidirectio (None, 16)                4672      
_________________________________________________________________
dense_81 (Dense)             (None, 3)                 51        
Total params: 3,518,303
Trainable params: 518,003
Non-trainable params: 3,000,300
_____________________________________________

### Два года, 5 классов, weighted categorical_crossentropy loss

In [128]:
years_train=[2007,2008]
years_test=[2007, 2008]
test_index=6

vectorizer = TextVectorization(max_tokens=10000, output_sequence_length=300)

train_data, train_labels = FormTrainData(years_train[0], test_index, 7)
test_data, test_labels = ReadData(years_test[0], test_index)
train_data_2, train_labels_2 = FormTrainData(years_train[1], test_index, 7)
test_data_2, test_labels_2 = ReadData(years_test[1], test_index)
train_data = pd.concat((train_data,train_data_2),axis=0)
train_labels = pd.concat((train_labels,train_labels_2),axis=0)
test_data = pd.concat((test_data,test_data_2),axis=0)
test_labels = pd.concat((test_labels,test_labels_2),axis=0)

# test_data, test_labels = ReadData(year_test, test_index)
# test_labels = test_labels.apply(lambda x: 1 if x == 3 else (0 if x < 3 else 2))


test_labels = test_labels - 1
train_labels = train_labels - 1

vectorizer.adapt(train_data.values)
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))
nb_words = len(voc)+2
train_data = vectorizer(np.array([[s] for s in train_data])).numpy()
test_data = vectorizer(np.array([[s] for s in test_data])).numpy()
embedding_matrix = BuildEmbeddingMatrix(nb_words, word_index)
print(nb_words, train_data.shape[1])

10001 300


In [129]:
class_weights = class_weight.compute_class_weight('balanced', np.unique(test_labels), test_labels)
class_weights

array([2.85306705, 3.64963504, 2.36966825, 0.835771  , 0.36271309])

In [130]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(64))(x)
outputs = Dense(5, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function='categorical_crossentropy', class_weight=class_weight, epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_72 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_71 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_89 (Bidirectio (None, 128)               186880    
_________________________________________________________________
dense_82 (Dense)             (None, 5)                 645       
Total params: 3,187,825
Trainable params: 187,525
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 3750 steps
Epoch 1/10
3750/3750 [==============================] - 69s 18ms/step - loss: 1.2022 - f1: 0.2354 - accuracy: 0.5509
Epoch 2/10
3750/3750 [==============================] - 68s 18ms/step - loss: 1.

In [131]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(64))(x)
outputs = Dense(5, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function='categorical_crossentropy', class_weight=[1,1,1], epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_73 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_72 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_90 (Bidirectio (None, 128)               186880    
_________________________________________________________________
dense_83 (Dense)             (None, 5)                 645       
Total params: 3,187,825
Trainable params: 187,525
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 3750 steps
Epoch 1/10
3750/3750 [==============================] - 69s 19ms/step - loss: 1.2001 - f1: 0.2354 - accuracy: 0.5508
Epoch 2/10
3750/3750 [==============================] - 68s 18ms/step - loss: 1.

In [132]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(128))(x)
outputs = Dense(5, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function='categorical_crossentropy', class_weight=class_weight, epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_74 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_73 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_91 (Bidirectio (None, 256)               439296    
_________________________________________________________________
dense_84 (Dense)             (None, 5)                 1285      
Total params: 3,440,881
Trainable params: 440,581
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 3750 steps
Epoch 1/10
3750/3750 [==============================] - 87s 23ms/step - loss: 1.2104 - f1: 0.2354 - accuracy: 0.5503
Epoch 2/10
3750/3750 [==============================] - 87s 23ms/step - loss: 1.

In [134]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = LSTM(128)(x)
outputs = Dense(5, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function='categorical_crossentropy', class_weight=class_weight, epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_76 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_75 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
lstm_100 (LSTM)              (None, 128)               219648    
_________________________________________________________________
dense_86 (Dense)             (None, 5)                 645       
Total params: 3,220,593
Trainable params: 220,293
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 3750 steps
Epoch 1/10
3750/3750 [==============================] - 45s 12ms/step - loss: 1.2316 - f1: 0.2375 - accuracy: 0.5512
Epoch 2/10
3750/3750 [==============================] - 45s 12ms/step - loss: 1.

In [135]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(64))(x)
x = Dense(32, activation='relu')(x)
outputs = Dense(5, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function='categorical_crossentropy', class_weight=class_weight, epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_77 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_76 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_92 (Bidirectio (None, 128)               186880    
_________________________________________________________________
dense_87 (Dense)             (None, 32)                4128      
_________________________________________________________________
dense_88 (Dense)             (None, 5)                 165       
Total params: 3,191,473
Trainable params: 191,173
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 3750 steps
Epoch 1/10
3750/3750 [==============================] - 69s 18ms/

In [136]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(128))(x)
x = Dense(64, activation='relu')(x)
outputs = Dense(5, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function='categorical_crossentropy', class_weight=class_weight, epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_78 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_77 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_93 (Bidirectio (None, 256)               439296    
_________________________________________________________________
dense_89 (Dense)             (None, 64)                16448     
_________________________________________________________________
dense_90 (Dense)             (None, 5)                 325       
Total params: 3,456,369
Trainable params: 456,069
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 3750 steps
Epoch 1/10
3750/3750 [==============================] - 88s 23ms/

In [137]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Bidirectional(LSTM(32))(x)
outputs = Dense(5, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function='categorical_crossentropy', class_weight=class_weight, epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_79 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_78 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_94 (Bidirectio (None, None, 256)         439296    
_________________________________________________________________
bidirectional_95 (Bidirectio (None, 64)                73984     
_________________________________________________________________
dense_91 (Dense)             (None, 5)                 325       
Total params: 3,513,905
Trainable params: 513,605
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 3750 steps
Epoch 1/10
3750/3750 [==============================] - 161s 43ms

In [138]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Bidirectional(LSTM(64))(x)
outputs = Dense(5, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function='categorical_crossentropy', class_weight=class_weight, epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_80 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_79 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_96 (Bidirectio (None, None, 256)         439296    
_________________________________________________________________
bidirectional_97 (Bidirectio (None, 128)               164352    
_________________________________________________________________
dense_92 (Dense)             (None, 5)                 645       
Total params: 3,604,593
Trainable params: 604,293
Non-trainable params: 3,000,300
_________________________________________________________________
  ...
    to  
  ['...']
Train for 3750 steps
Epoch 1/10
3750/3750 [==============================] - 164s 44ms

In [139]:
inputs = keras.Input(shape=(None,), dtype="int64")       
x = Embedding(nb_words, 300,input_length=train_data.shape[1], weights=[embedding_matrix],trainable=False)(inputs)
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Bidirectional(LSTM(32, return_sequences=True))(x)
x = Bidirectional(LSTM(8, return_sequences=False))(x)
outputs = Dense(5, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="model")
test_model(model=model, loss_function='categorical_crossentropy', class_weight=class_weight, epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_81 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_80 (Embedding)     (None, None, 300)         3000300   
_________________________________________________________________
bidirectional_98 (Bidirectio (None, None, 256)         439296    
_________________________________________________________________
bidirectional_99 (Bidirectio (None, None, 64)          73984     
_________________________________________________________________
bidirectional_100 (Bidirecti (None, 16)                4672      
_________________________________________________________________
dense_93 (Dense)             (None, 5)                 85        
Total params: 3,518,337
Trainable params: 518,037
Non-trainable params: 3,000,300
_____________________________________________